In [ ]:
import pandas as pd 
#pd.set_option('compute.use_numexpr', False)
import numpy as np 
from collections import defaultdict
import re
import sys
from sklearn.model_selection import train_test_split
import gzip
import yaml
import requests
import csv
import re
from tqdm import tqdm
from itertools import product
import json
import pickle
import matplotlib.pyplot as plt
from itertools import cycle, islice

In [ ]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals
import sys
# reload(sys)
# sys.setdefaultencoding('utf8')
import os
import json
import string
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import random
from numpy import array
import re
import pickle
#from pattern.text.en import singularize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
#import enchant
from nltk.stem import LancasterStemmer
import nltk
from nltk import word_tokenize
from nltk.util import ngrams
from nltk.corpus import stopwords
from sklearn.metrics import f1_score
import operator
from collections import Counter
from nltk.corpus import wordnet as wn
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
from collections import defaultdict
import math
import csv

import pandas as pd 
import numpy as np 
from collections import defaultdict
import re
import sys
from sklearn.model_selection import train_test_split
import gzip

import csv
import pandas as pd 
import numpy as np 
from collections import defaultdict
import re
import sys
from sklearn.model_selection import train_test_split
import datetime
import matplotlib.pyplot as plt
import time
import numpy as np
from pprint import pprint

import spacy  # For preprocessing
import nltk
import string
from nltk.corpus import stopwords
#import preprocessor as p  #pip install tweet-preprocessor
import logging  # Setting up the loggings to monitor gensim
from nltk.stem import WordNetLemmatizer
from string import punctuation as punc

import gensim
import gensim.corpora as corpora
#from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import gensim.models as gsm
from gensim.test.utils import datapath

import regex


import torch
#import torchtext
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd
import torchvision.transforms as transforms
from torch.autograd import Variable

import spacy
import string

from nltk.collocations import BigramCollocationFinder, BigramAssocMeasures
from nltk.corpus import stopwords

torch.manual_seed(1)





In [ ]:
predicted_df = pd.read_csv( 'data/new_predicted_35k_with_para_with_weak_gt.csv',low_memory=False) #(28189, 6)

predicted_df #852799 rows × 11 columns


In [ ]:
###Baseline
df = predicted_df[['id','ad_creative_body', 'funding_entity', 'weak_stance', 'gt_stance']] #extracted 4 columns

In [ ]:
# #take the rows where weak label (weak_stance) is not NAN. We will use this to train our supervised model
df_train = df[df['weak_stance'].notna()] #5975 #train data
#df = df[df['EPS'].notna()]
#df_train = df_train.reset_index(drop=True) #5975 #train data
#print(df_train)
df_test = df.drop(df_train.index) #29348 # test data (no weak label)
#print(df_test)

In [ ]:
df_train = df_train.reset_index(drop=True) 

df_test = df_test.reset_index(drop=True) 

In [ ]:
df_trn = df_train.sample(frac = 0.80) #4780
#print(df_trn)
# rest of the 20% values
df_val = df_train.drop(df_trn.index) #1195
#print(df_val)

In [ ]:
df_trn = df_trn.reset_index(drop=True) 

df_val = df_val.reset_index(drop=True) 

In [ ]:
# df_trn.to_csv('data/baseline_train_fb_ad_cv7.csv', line_terminator='\r\n', encoding='utf-8', index = False) 
# df_val.to_csv('data/baseline_val_fb_ad_cv7.csv', line_terminator='\r\n', encoding='utf-8', index = False) 
# df_test.to_csv('data/baseline_test_fb_ad_cv7.csv', line_terminator='\r\n', encoding='utf-8', index = False)


In [ ]:
##Load train data for cv1
df_trn = pd.read_csv( 'data/baseline_train_fb_ad_cv3.csv',low_memory=False)

In [ ]:
#df_trn

In [ ]:
#tokenization
tok = spacy.load('en_core_web_sm')
def tokenize (text):
    text = re.sub(r"[^\x00-\x7F]+", " ", text)
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]') # remove punctuation and numbers
    nopunct = regex.sub(" ", text.lower())
    return [token.text for token in tok.tokenizer(nopunct)]

In [ ]:
#count number of occurences of each word
counts = Counter()
for index, row in df_trn.iterrows():
    counts.update(tokenize(row['ad_creative_body']))

In [ ]:
#deleting infrequent words
print("num_words before:",len(counts.keys()))
for word in list(counts):
    if counts[word] < 2:
        del counts[word]
print("num_words after:",len(counts.keys()))

In [ ]:
#creating vocabulary on train data
vocab2index = {"":0, "UNK":1}
words = ["", "UNK"]
for word in counts:
    vocab2index[word] = len(words)
    words.append(word)

In [ ]:
print(len(words))

In [ ]:
def encode_sentence(text, vocab2index, N=300):
    tokenized = tokenize(text)
    encoded = np.zeros(N, dtype=int)
    enc1 = np.array([vocab2index.get(word, vocab2index["UNK"]) for word in tokenized])
    length = min(N, len(enc1))
    encoded[:length] = enc1[:length]
    return encoded, length

In [ ]:
df_trn['encoded'] = df_trn['ad_creative_body'].apply(lambda x: np.array(encode_sentence(x,vocab2index )))
#df_trn.head()

In [ ]:
df_val = pd.read_csv( 'data/baseline_val_fb_ad_cv3.csv',low_memory=False)

In [ ]:
df_val['encoded'] = df_val['ad_creative_body'].apply(lambda x: np.array(encode_sentence(x,vocab2index )))
#df_val.head()

In [ ]:
X_train = list(df_trn['encoded'])
y_train = list(df_trn['weak_stance'])


X_valid = list(df_val['encoded'])
y_valid = list(df_val['weak_stance'])


In [ ]:
#library imports
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import re
import spacy
from collections import Counter
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import string
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from sklearn.metrics import mean_squared_error

In [ ]:
class FbAd(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.y = Y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx][0].astype(np.int32)), self.y[idx], self.X[idx][1]

In [ ]:
train_ds = FbAd(X_train, y_train)
valid_ds = FbAd(X_valid, y_valid)

In [ ]:
def train_model(model, epochs=10, lr=0.001):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    best_valid_loss = float('inf')
    for i in range(epochs):
        print("starting of epochs : ", i )
        model.train()
        sum_loss = 0.0
        total = 0
        correct = 0
        for x, y, l in train_dl:
            #print(type(x), x)
            #print(type(y), y )
            new_y = [] # to handle multi-label
            for j in range(0, len(y)):
                if len(y[j]) > 1:
                    new_y.append(int(y[j][0]))
                else:
                    new_y.append(int(y[j]))
                    
            #print(new_y)
            #x = x.long()
            y = torch.tensor(new_y)
            #print(y)
            y_pred = model(x, l)
            #print(type(y_pred), y_pred)
            optimizer.zero_grad()
            loss = F.cross_entropy(y_pred, y)
            #print("train loss", loss.item())
            loss.backward()
            optimizer.step()
            pred_train = torch.max(y_pred, 1)[1]
            correct += (pred_train == y).float().sum()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc = validation_metrics(model, val_dl)
        if val_loss <= best_valid_loss:
            best_valid_loss = val_loss
            print("best model saved in epoch :", i )
            torch.save(model.state_dict(), 'data/glove_lstm_fb_ad_model_cv3_'+str(i)+'.pt')
        if i % 5 == 1:
            print("train loss %.3f,  train accuracy %.3f , val loss %.3f, val accuracy %.3f  " % (sum_loss/total, correct/total, val_loss, val_acc))

def validation_metrics (model, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    sum_rmse = 0.0
    for x, y, l in valid_dl:
#         x = x.long()
#         y = y.long()
        new_y = [] # to handle multi-label
        for j in range(0, len(y)):
            if len(y[j]) > 1:
                new_y.append(int(y[j][0]))
            else:
                new_y.append(int(y[j]))

        #print(new_y)
        #x = x.long()
        y = torch.tensor(new_y)
        y_hat = model(x, l)
        #print(y_hat, torch.max(y_hat), torch.max(y_hat, 1))
        loss = F.cross_entropy(y_hat, y)
        pred = torch.max(y_hat, 1)[1]
        correct += (pred == y).float().sum()
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
        
    return sum_loss/total, correct/total


In [ ]:
batch_size = 32
vocab_size = len(words)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(valid_ds, batch_size=batch_size)

In [ ]:
def load_glove_vectors(glove_file="data/glove.6B.300d.txt"):
    """Load the glove word vectors"""
    word_vectors = {}
    with open(glove_file) as f:
        for line in f:
            split = line.split()
            word_vectors[split[0]] = np.array([float(x) for x in split[1:]])
    return word_vectors

In [ ]:
def get_emb_matrix(pretrained, word_counts, emb_size = 300):
    """ Creates embedding matrix from word vectors"""
    vocab_size = len(word_counts) + 2
    vocab_to_idx = {}
    vocab = ["", "UNK"]
    W = np.zeros((vocab_size, emb_size), dtype="float32")
    W[0] = np.zeros(emb_size, dtype='float32') # adding a vector for padding
    W[1] = np.random.uniform(-0.25, 0.25, emb_size) # adding a vector for unknown words 
    vocab_to_idx["UNK"] = 1
    i = 2
    for word in word_counts:
        if word in word_vecs:
            W[i] = word_vecs[word]
        else:
            W[i] = np.random.uniform(-0.25,0.25, emb_size)
        vocab_to_idx[word] = i
        vocab.append(word)
        i += 1   
    return W, np.array(vocab), vocab_to_idx

In [ ]:
word_vecs = load_glove_vectors()
pretrained_weights, vocab, vocab2index = get_emb_matrix(word_vecs, counts)

In [ ]:
# class LSTM_glove_vecs(torch.nn.Module) :
#     def __init__(self, vocab_size, embedding_dim, hidden_dim, glove_weights) :
#         super().__init__()
#         self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
#         self.embeddings.weight.data.copy_(torch.from_numpy(glove_weights))
#         self.embeddings.weight.requires_grad = False ## freeze embeddings
#         self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
#         self.linear = nn.Linear(hidden_dim, 4) #change based on number of labels
#         self.dropout = nn.Dropout(0.2)
        
#     def forward(self, x,l):
#         #print(x.size)
#         x = self.embeddings(x)
#         x = self.dropout(x)
#         lstm_out, (ht, ct) = self.lstm(x)
#         return self.linear(ht[-1])


class LSTM_glove_vecs(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim, glove_weights) :
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.embeddings.weight.data.copy_(torch.from_numpy(glove_weights))
        self.embeddings.weight.requires_grad = False ## freeze embeddings
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, num_layers=1, bidirectional=True)
        self.linear = nn.Linear(hidden_dim, 4) #change based on number of labels
        self.dropout = nn.Dropout(0.2)
        
    def init_hidden(self, batch_size):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        if torch.cuda.is_available():
            return (torch.zeros(2, batch_size, 150).cuda(),
                    torch.zeros(2, batch_size, 150).cuda())
        return (torch.zeros(2, batch_size, 150),
                torch.zeros(2, batch_size, 150))
    
    def forward(self, x,l):
        #print(x.size)
        # reset the LSTM hidden state. Must be done before you run a new batch. Otherwise the LSTM will treat
        # a new batch as a continuation of a sequence
        self.hidden = self.init_hidden(len(x))
        x = self.embeddings(x)
        x = self.dropout(x)
        lstm_out, self.hidden = self.lstm(x, self.hidden)
        #print(self.hidden, self.hidden[0], self.hidden[1])
        return self.linear(self.hidden[0][-1])

In [ ]:
print(vocab_size) 

In [ ]:
model = LSTM_glove_vecs(vocab_size, 300, 150, pretrained_weights)

In [ ]:
train_model(model, epochs=20, lr=0.01)

In [ ]:
#for test prediction
df_test = pd.read_csv( 'data/baseline_test_fb_ad_cv3.csv',low_memory=False)

In [ ]:
df_test['encoded'] = df_test['ad_creative_body'].apply(lambda x: np.array(encode_sentence(x,vocab2index )))
#df_test.head()

In [ ]:
X_test = list(df_test['encoded'])
y_test = list(df_test['weak_stance'])

In [ ]:
test_ds = FbAd(X_test, y_test)

In [ ]:
test_dl = DataLoader(test_ds, batch_size=1)

In [ ]:
def test_prediction (model, test_dl):
    y_pred = []
    model.eval()
    for x, y, l in test_dl:
        #print(l)
        y_hat = model(x, l)
        pred = torch.max(y_hat, 1)[1]
        y_pred.append(pred.item())
    return y_pred
        

In [ ]:
model.load_state_dict(torch.load('data/glove_lstm_fb_ad_model_cv3_2.pt')) 
test_pred = test_prediction(model, test_dl)

In [ ]:
print(len(test_pred))
print(test_pred[0])

In [ ]:
#test_pred

In [ ]:
df_test['pred_stance'] = test_pred

In [ ]:
##Baseline (Glove + LSTM) -- accuracy
df_b = df_test[['id', 'pred_stance', 'gt_stance']] #extracted 3 columns
#print(df_b) #29348
df_b1= df_b.dropna()#dropping nan value
#print(df_b1) #385
df_b1 = df_b1[df_b1.gt_stance != '-1'] #dropping rows with truth value -1
df_b1 = df_b1.reset_index(drop = True)
#print(df_b1) #279
count_label = 0

for i in range (0, df_b1.shape[0]):
    #print(df_b1.gt_stance[i][0], type(df_b1.gt_stance[i]))

    if df_b1.pred_stance[i] == int(df_b1.gt_stance[i][0]): #count label those are same for both ground truth and prediction
        count_label = count_label + 1

print('count_label', count_label) 
print('accuracy of baseline : ', (count_label/df_b1.shape[0]) ) 

#Bi-LSTM
#0.4982078853046595, 0.5340501792114696,  0.5232974910394266, 0.5734767025089605, 0.5483870967741935 

# mean = 0.5355

#LSTM
#0.6093189964157706,  0.1222222222222222,  0.5483870967741935, 0.07885304659498207, 0.078853046594982072, 0.07885304659498207


##Baseline (Glove + LSTM) -- Macro avg F1 score
from sklearn.metrics import f1_score
#print(df_b1.gt_stance.values)
new_y = [] # to handle multi-label
for j in range (0, df_b1.shape[0]):
    if len(df_b1.gt_stance[j]) > 1:
        new_y.append(int(df_b1.gt_stance[j][0]))
    else:
        new_y.append(int(df_b1.gt_stance[j]))

#print(new_y)
#x = x.long()
print('Macro-avg F1 score of baseline : ', f1_score(new_y, df_b1.pred_stance.values, average='macro'))

#Bi-LSTM
# 0.363782832520845 , 0.38692407749292423, 0.40802426122331525, 0.5013154244058546, 0.4211751317915702

# mean = 0.4162

#LSTM
#0.2171822518408783, 0.05145827826240197, 0.37591626828914965, 0.033444816053511704, 0.04141898086534764,  0.039739500265816054


